# Scraper
Scrapes odds from Unibet, predictions from fivethirtyeight and shows you the best bets.

Start by starting the browser. Then follow the cells one time to click the cookie-bar.

In [ ]:
# Start our headless (no GUI) browser - Run this only 1 time
from selenium import webdriver
browser = webdriver.Firefox()

In [ ]:
# Quit the browser - Run this only when you're done
browser.quit()

## `Run All Below` cells from here
Of course set the league you want to scrape first.

In [ ]:
# choose a league (0-5)
currentLeague = 4

In [ ]:
# create the url matrix
import pandas as pd
urlUnibet = ['https://www.unibet.eu/betting#filter/football/netherlands/eredivisie',
              'https://www.unibet.eu/betting#filter/football/spain/la_liga',
              'https://www.unibet.eu/betting#filter/football/germany/bundesliga',
              'https://www.unibet.eu/betting#filter/football/england/premier_league',
              'https://www.unibet.eu/betting#filter/football/france/ligue_1',
              'https://www.unibet.eu/betting#filter/football/italy/serie_a']
url538 = ['https://projects.fivethirtyeight.com/soccer-predictions/eredivisie/',
           'https://projects.fivethirtyeight.com/soccer-predictions/la-liga/',
           'https://projects.fivethirtyeight.com/soccer-predictions/bundesliga/',
           'https://projects.fivethirtyeight.com/soccer-predictions/premier-league/',
           'https://projects.fivethirtyeight.com/soccer-predictions/ligue-1/',
           'https://projects.fivethirtyeight.com/soccer-predictions/serie-a/']
dfUrl = pd.DataFrame(data={'urlUnibet': urlUnibet, 'url538': url538})
#dfUrl

## Odds from unibet

In [ ]:
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
# Go to the URL
url = dfUrl.loc[currentLeague, 'urlUnibet']
print('Going to:', url)
browser.get(url)

In [ ]:
# Wait until the accept-cookies button is present
try:
    element = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyButtonAccept"))
    )

    # When its present, let's wait another sec
    sleep(1)

    # Click on it
    element.click()
except TimeoutException:
    # Not found, but maybe already clicked
    pass
finally:
    # Wait until the cookie bar is gone
    WebDriverWait(browser, 10).until( EC.invisibility_of_element_located((By.ID, "CybotCookiebotDialogBodyButtons")))
    
    # Just to be sure
    sleep(1)

In [ ]:
# Wait for a dropdown bar
WebDriverWait(browser, 10).until( EC.presence_of_all_elements_located((By.CLASS_NAME, "KambiBC-collapsible-container")) )

# No hurry please
sleep(np.pi)

# Find the dropdown bars, NOT the already expanded one
dropdowns = browser.find_elements_by_css_selector(".KambiBC-event-groups .KambiBC-collapsible-container:not(.KambiBC-expanded)")

# Click on them
for dropdown in dropdowns:
    dropdown.click()
    sleep(np.pi)

In [ ]:
# Now get the beautiful source code
soup = BeautifulSoup(browser.page_source, 'html5lib')

# Just wait some more maybe
sleep(np.sqrt(np.exp(1)))

In [ ]:
# search for matches in the soup
matches = soup.find(class_="KambiBC-event-groups").find_all("li", class_="KambiBC-event-item")

# print amount found. Also remember this amount for getting the same amout from 538
numberOfMatches = len(matches)
print("Number of matches found: ", numberOfMatches)

In [ ]:
# Create empty list
jobs = []

for match in matches:
    # Create empty dictionary
    job = {}
    
    # Find info on the page
    #job["date"] = match.find(class_="KambiBC-event-item__start-time--date").text # <- Doesnt work..?
    teams = match.find_all(class_="KambiBC-event-participants__name")
    job["home_team"] = teams[0].text
    job["away_team"] = teams[1].text
    odds = match.find_all(class_="KambiBC-mod-outcome__odds")
    job["odd_home_win"] = odds[0].text
    job["odd_tie"] = odds[1].text
    job["odd_away_win"] = odds[2].text
    
    # Add to the list
    jobs.append(job)

In [ ]:
# Make a data frame
dfOdds = pd.DataFrame(jobs)

# Show
#dfOdds

### Data transformation

In [ ]:
# Change team names to lowercase
dfOdds["home_team"] = dfOdds["home_team"].str.lower()
dfOdds["away_team"] = dfOdds["away_team"].str.lower()

# Remove all accents
dfOdds["home_team"] = dfOdds["home_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')
dfOdds["away_team"] = dfOdds["away_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')

# Change the team names so that they match the ones in the 538 data frame
changes_nl = {'fc groningen': 'groningen',
              'fc twente': 'twente',
              'sc heerenveen': 'heerenveen',
              'fc utrecht': 'utrecht',
              'fc emmen': 'emmen'}
changes_de = {'bayer leverkusen': 'leverkusen',
             'borussia monchengladbach': 'gladbach',
             'vfl wolfsburg': 'wolfsburg',
             'borussia dortmund': 'dortmund',
             'augsburg': 'fc ausburg'}
changes_es = {'deportiva las palmas': 'las palmas',
             'fc barcelona': 'barcelona'}
changes_en = {}
changes_fr = {'saint-etienne': 'st etienne',
             'paris sg': 'psg'}
changes_it = {'hellas verona': 'verona'}
changes = {**changes_nl, **changes_de, **changes_es, **changes_en, **changes_fr, **changes_it}
for old,new in changes.items():
    # Replace!
    dfOdds["home_team"] = dfOdds["home_team"].str.replace(old, new)
    dfOdds["away_team"] = dfOdds["away_team"].str.replace(old, new)

# Make 3-letter code names
dfOdds["home_code"] = dfOdds["home_team"].str[:3]
dfOdds["away_code"] = dfOdds["away_team"].str[:3]

# Convert to numbers
dfOdds["odd_home_win"] = pd.to_numeric(dfOdds["odd_home_win"])
dfOdds["odd_away_win"] = pd.to_numeric(dfOdds["odd_away_win"])
dfOdds["odd_tie"] = pd.to_numeric(dfOdds["odd_tie"])

# Show
dfOdds

## Predictions from five thirty eight

In [ ]:
# so easy site, just use a basic scraper
from lxml import html
from lxml.cssselect import CSSSelector
import requests
import re
import pandas as pd

In [ ]:
# Go to the URL
url = dfUrl.loc[currentLeague, 'url538']
page = requests.get(url)

# Get the source code
tree = html.fromstring(page.content)

In [ ]:
# it shows all matches of the season here. Let's stick to the same number as from Unibet
matches = tree.cssselect('.games-container.upcoming .match-container')[:numberOfMatches]
print("Number of matches found: ", len(matches))

In [ ]:
# prepare empty data frame
cols = ['date', 'home_team', 'away_team', 'home_win', 'tie', 'away_win']
df538 = pd.DataFrame(columns=cols)

# fill data frame with match info
for idx in range(len(matches)):
    match = matches[idx]
    
    df538.at[idx, 'date'] = match.cssselect(".date div")[0].text
    df538.at[idx, 'home_team'] = match.cssselect(".match-top .name")[0].text
    df538.at[idx, 'home_win'] = match.cssselect(".match-top .prob")[0].text
    df538.at[idx, 'tie'] = match.cssselect(".tie-prob div")[0].text
    df538.at[idx, 'away_team'] = match.cssselect(".match-bottom .name")[0].text
    df538.at[idx, 'away_win'] = match.cssselect(".match-bottom .prob")[0].text
    
#df538

### Some data transformation

In [ ]:
# Turn percentages into floats
df538['home_win'] = pd.to_numeric(df538['home_win'].str.replace("%",""))/100
df538['tie'] = pd.to_numeric(df538['tie'].str.replace("%",""))/100
df538['away_win'] = pd.to_numeric(df538['away_win'].str.replace("%",""))/100

# Parse the date column (M/d to d/M)
df538['date'] = df538['date'].apply(lambda d: '{l[1]}-{l[0]}'.format(l=d.split('/')))

# Change team names to lowercase
df538["home_team"] = df538["home_team"].str.lower()
df538["away_team"] = df538["away_team"].str.lower()

# Remove all accents
df538["home_team"] = df538["home_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')
df538["away_team"] = df538["away_team"].str.normalize('NFKD').str.encode(encoding='ascii',errors='ignore').str.decode('utf-8')

# Make 3-letter code names
df538["home_code"] = df538["home_team"].str[:3]
df538["away_code"] = df538["away_team"].str[:3]

# SHow
df538

# Merge
First check if team names are missed in the data transformation.

If the table below is empty, then it's alright.

In [ ]:
# check for missing matches for Unibet
dfCheck = df538.merge(dfOdds, how="outer", on=["home_code", "away_code"], indicator=True)
dfCheck[dfCheck._merge != 'both']

In [ ]:
# Merge by 3-letter codes
dfMerge = df538.merge(dfOdds, how="inner", on=["home_code", "away_code"], suffixes=('', '_uni'))

# print
print('Number of matches succesfully merged:', len(dfMerge))

# Analyse

In [ ]:
# Calculate the expected profits for win, tie and loss
dfMerge["expect_home"] = dfMerge["home_win"] * dfMerge["odd_home_win"]
dfMerge["expect_away"] = dfMerge["away_win"] * dfMerge["odd_away_win"]
dfMerge["expect_tie"] = dfMerge["tie"] * dfMerge["odd_tie"]

# Set threshold
threshold = 1.15

# Show
theGames = dfMerge.query('expect_home>@threshold or expect_away>@threshold or expect_tie>@threshold').copy()
columns = ["date", "home_team", "away_team", "expect_home", "expect_tie", "expect_away"]
display(theGames[columns])

# Print to CSV

In [ ]:
# first add the date of today
from datetime import datetime
now = datetime.now()
theGames['date_of_bet'] = '%s-%s-%s' % (now.day, now.month, now.year)
theGames['league'] = url.split('/')[-2]

In [ ]:
# append to book.csv
theGames = theGames[['date', 'home_team', 'away_team', 'league', 'home_win', 'tie', 'away_win',\
                     'odd_home_win', 'odd_tie', 'odd_away_win', 'expect_home', 'expect_tie', 'expect_away', 'date_of_bet']]

#theGames.to_csv('./book.csv', index=False, sep=';', float_format='%.3f', decimal=',')
#theGames.to_csv('./book.csv', index=False, sep=';', float_format='%.3f', decimal=',', mode='a', header=False)